ссылка на гитхаб репозиторий

https://github.com/theorcos239/async_ppo.git



# Коротко о структуре

model.py - файл, в котором хранится класс основной действующей модели: свёрточная нейронная сеть 
с двумя головами для actor и critic

learner.py - файл, содержащий класс, реализующий PPO, с соответствующими методами по сбору роллаутов и обучению

rollout_buffer.py - реализация класса роллаута

worker_process.py - функция процесса асинхронного воркера для сбора роллаутов

learner_process.py - процесс обучения центральной модели 






# Параметры эксперимента

workers_count = 4

rollout_size = 2048

lr = 3e-4

batch_size = 256

n_epochs = 4
                        
gamma = 0.99 

delta = 0.95

clip_eps = 0.2
                              
entropy_coef = 0.01 

value_coef = 0.5

Обучение проходило в течение 3-5 часов на эксперимент до плато на графике reward(полное обучение). Требуемые графики(все 7) находятся в ppo_plots. В результатах только самое примечательное.


# Результаты

На всех графиках время измеряется в минутах.

<img src="./ppo_plots/reward_vs_time.png">

На этом графике представлен также результат обучения синхронной версии. Видно, что синхронная модель обучается сильно медленне, в частности, из-за простоя критика. Синхронная версия обучалачь около 500 минут и только там вышла на плато (кусок графика обрезан, иначе бы асинхронные графики сжались и стали непонятными).

В среднем модели с параматром frequency = 1 обучаются быстрее, потому что обновление весов чаще => меньше устаревающей информации. Поэтому модель тренируется на актульных данных.

По аналогичной причине в среднем чем меньше max_queue_sizeб тем данные меньше застаиваются => опять лучше. Но это не всегда так, потому что набор параметров (10,10) показал себя довольно достойно, но это скорее выброс.

<img src="./ppo_plots/reward_vs_envsteps.png">

Аналогичная история для графика reward от envsteps. Потому что env_teps очень сильно зависит от времени, а воркеров одно и тоже количество во всех экспериментах. Иногда, конечно, локальные сдвиги графиков по сравнению с предыдущим просиходят из-за простоев. Но в целом закономерность та же.

<img src="./ppo_plots/updates_vs_time.png">

Выше тоже закономерный результат. Зависимость примерно линейная с момента, когда все процессы разогнались и стабилизировались. По такой же причине на графике env_steps от времени тоже линейный.

<img src="./ppo_plots/qsize_vs_time.png">

Из графика зависимости размера очереди от времени видно, что очередь во всех случаях обычно полна и редко изменяется на 1. Наверное, это связано с тем, что нейронная сеть всё таки не успевает обрабатывать всё, что приходит от воркеров. 

<img src="./ppo_plots/versiondiff_vs_time.png">

Видно, что постепенно версия роллаутов догоняет версию центральной нейросети, а затем держится примерно на одном уровне. И отставание в итоге не очень большое. Поэтому данные не очень старые и алгоритм сходится.